In [1]:
from tqdm import tqdm
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
import os
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder,StandardScaler
from sklearn.metrics import precision_score

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Flatten,CuDNNLSTM, LSTM,RepeatVector
from keras.layers import TimeDistributed,Dropout, LeakyReLU, ReLU , BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import RMSprop, Adam,SGD
from tensorflow.keras.models import load_model

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
db_daesin = pymysql.connect(host='221.148.138.91', port=3306, user='remote', password='1234', db='dbstock')
cursor_daesin = db_daesin.cursor()
cursor_daesin.execute('use dbstock;')

db_kiwoom = pymysql.connect(host='61.77.150.183', port=3306, user='a1_user',password='4560',db='kiwoom',charset='utf8')
cursor_kiwoom = db_kiwoom.cursor()
cursor_kiwoom.execute('use kiwoom;')

0

In [5]:
def make_folder():
    
    folders = ['./csv','./csv/raw','./csv/filtered','./numpy',
               './numpy/B','./numpy/C','./numpy/D',
               './numpy/E','./numpy/concat','./model']
    
    for names in folders :
        
        if not os.path.exists(names):
            os.mkdir(names)           


def extract_dates(path = 'raw'):
    
#     path = './csv'
    file_list = os.listdir('./csv/'+ path)
    
    date_list = []
    for i in file_list:
        
        date_list.append(i.split('_')[0])
    
    return date_list

def daesin_main(code, date,db=db_daesin):    

    sql = 'select * from {};'.format(code)
    df_dae = pd.read_sql(sql, db)
    df_dae  = df_dae.set_index('체결시간')
    df_dae= df_dae[['매수잔량','매도잔량','체결강도']]
    df_dae = df_dae.loc[date]
    df_dae.drop(df_dae.between_time('15:20','15:29').index, inplace=True)
    df_dae.drop(df_dae.between_time('15:31','15:32').index, inplace=True)
    df_dae.drop(df_dae.at_time('23:59').index, inplace=True)
    
    
    return  df_dae

def daesin_kospi(date,db=db_daesin):  

    sql_kospi = 'select * from u001;'
    df_kospi = pd.read_sql(sql_kospi, db)
    df_kospi = df_kospi.set_index('체결시간')[['코스피지수']]

    df_kospi= df_kospi.loc[date]
    df_kospi.drop(df_kospi.between_time('15:20','15:29').index, inplace=True)
    df_kospi.drop(df_kospi.between_time('15:31','15:32').index, inplace=True)
    df_kospi.drop(df_kospi.at_time('23:59').index, inplace=True)    
    globals()['df_kospi_{}'.format(date.replace('-',''))] = df_kospi

    return globals()['df_kospi_{}'.format(date.replace('-',''))]


def daesin_issued(code,db=db_daesin):
    
    code2 = code.replace(code[0], code[0].upper())
    sql_issued = 'select * from z001;'
    df_issued = pd.read_sql(sql_issued, db)
    df_issued=  df_issued[df_issued.종목코드 == code2][['발행주식수']]
    
    globals()['{}_issued'.format(code)] = df_issued.values[0][0]
    
    return globals()['{}_issued'.format(code)]  

def kiwoom(code, date,db=db_kiwoom) : 
    
    sql_kiwoom = 'select * from {};'.format(code)
    df_kiwoom = pd.read_sql(sql_kiwoom, db)
    df_kiwoom  = df_kiwoom.set_index('체결시간')
    df_kiwoom= df_kiwoom.loc[date]
    df_kiwoom.rename(columns={'현재가':'종가'},inplace=True)
    df_kiwoom.drop('누적거래대금', axis=1, inplace=True)

    return df_kiwoom

def merge_df(date):
    
    df_dae, df_kiwoom, df_kospi= daesin_main(code, date,db=db_daesin),kiwoom(code, date,db=db_kiwoom), globals()['df_kospi_{}'.format(date.replace('-',''))]
    
    df_stock_temp = pd.merge(df_dae, df_kiwoom, left_index = True, right_index=True, how='inner')
    df_stock = pd.merge(df_stock_temp,df_kospi,
                        left_index = True, right_index=True, how='inner')
    df_stock = df_stock.apply(lambda x : abs(x))
    merged_df = df_stock.copy()
    
    return merged_df


def preprocessing(code, date):
    
    merged_df = merge_df(date)
    issued =  globals()['{}_issued'.format(code)]
    
    merged_df['매수잔량'] = (merged_df['매수잔량']/issued)*100
    merged_df['매도잔량'] = (merged_df['매도잔량']/issued)*100
    merged_df['거래량'] = (merged_df['거래량']/issued)*100
    merged_df.loc[date + ' 15:30:00', '거래량'] = merged_df.at_time('15:30')['거래량'].values[0]/10

    merged_df['코스피지수'] = merged_df['코스피지수'].pct_change()
    merged_df['시가'] = merged_df['시가'].pct_change()
    merged_df['고가'] = merged_df['고가'].pct_change()
    merged_df['저가'] = merged_df['저가'].pct_change()
    
    drop_index = merged_df[(merged_df['체결강도']== 0)|(merged_df['매수잔량']== 0)|(merged_df['체결강도']== 0)].index
    merged_df.drop(drop_index, axis=0,inplace=True)
    
    merged_df.insert(3,'체결강도_pct',merged_df['체결강도'].pct_change())
    merged_df.insert(1,'매수잔량_pct',merged_df['매수잔량'].pct_change())
    merged_df.insert(3,'매도잔량_pct',merged_df['매도잔량'].pct_change())
    merged_df.insert(7,'거래량_pct',merged_df['거래량'].pct_change())
    merged_df.insert(10,'종가_pct', merged_df['종가'].pct_change())

    
    merged_df.rename(columns = {'시가' : '시가_pct','고가' : '고가_pct','저가' : '저가_pct','코스피지수' : '코스피_pct'},inplace=True)
    
    merged_df.dropna(axis=0, inplace=True)

    merged_df.insert(0, 'time_section', merged_df.index)
    merged_df.loc[merged_df.between_time('09:00','10:30').index, 'time_section'] = '09:00~10:30'
    merged_df.loc[merged_df.between_time('10:31','13:59').index, 'time_section'] = '10:31~13:59'
    merged_df.loc[merged_df.between_time('14:00','15:30').index, 'time_section'] = '14:00~15:30'
    
#     scaler = MinMaxScaler()
#     merged_df[['매수잔량','매수잔량_pct','매도잔량','매도잔량_pct','체결강도_pct','체결강도','거래량_pct']] = scaler.fit_transform(merged_df[['매수잔량',
#                                                                                                                    '매수잔량_pct','매도잔량','매도잔량_pct',
#                                                                                                                    '체결강도_pct','체결강도','거래량_pct']])
   
    
#     scaler = StandardScaler()
#     merged_df[['매수잔량','매수잔량_pct','매도잔량','매도잔량_pct','체결강도_pct','체결강도','거래량_pct']] = scaler.fit_transform(merged_df[['매수잔량',
#                                                                                                                    '매수잔량_pct','매도잔량','매도잔량_pct',
#                                                                                                                    '체결강도_pct','체결강도','거래량_pct']])
    merged_df.drop(merged_df[merged_df['매수잔량_pct']==np.inf].index, axis=0, inplace=True)
    merged_df.drop(merged_df[merged_df['매도잔량_pct']==np.inf].index, axis=0, inplace=True)
    
    
    merged_df.drop(['매수잔량','매도잔량','체결강도'], axis=1, inplace=True)
    merged_df.drop(['시가_pct','고가_pct','저가_pct'], axis=1, inplace=True)
    
#     merged_df = pd.get_dummies(merged_df)

    merged_df.drop('time_section',axis=1,inplace=True)
    
    merged_df['target'] = merged_df['종가'].pct_change(periods=offset)
    merged_df.drop('종가', axis=1, inplace=True)
    

    for i in merged_df.index:
        if merged_df.loc[i,'target'] >=0 :
            merged_df.loc[i,'target'] = 'PLUS'
        else :
            merged_df.loc[i,'target'] = 'MINUS'

    
    return merged_df

def split_sequence(timesteps, offset):
    
    sequence = preprocessing(code, date)
    
    x, y = list(), list()
    
    target = 'target'
    
    for i in range(len(sequence)):
        
        end_ix = i + timesteps
        
        if end_ix > len(sequence) -offset:
            break
            
        seq_x, seq_y = sequence.drop(target,axis=1).iloc[i:end_ix], sequence.iloc[end_ix + offset-1][target]
        
        x.append(seq_x)
        y.append(seq_y)
        
        
        
    return np.array(x), np.array(y)

def train_test_split():
    
    x, y = split_sequence(timesteps, offset)
    
    e = LabelEncoder()
    e.fit(y)
    y = e.transform(y)
   
    
    cutoff_rate=0.8
    cutoff_train = round(x.shape[0] * cutoff_rate)

    x_train, y_train = x[:cutoff_train], y[:cutoff_train]
    x_test, y_test = x[cutoff_train:], y[cutoff_train:]
    
    return x_train, y_train, x_test, y_test


def filter_csv(timesteps, offset):
    
    date_list = extract_dates(path = 'raw')
    
    for date in date_list :
        
        df_raw = pd.read_csv('./csv/raw/{}_merged_groupping.csv'.format(date), index_col=0)
        raw_code_list = df_raw['종목코드']
        df_raw = df_raw.set_index('종목코드')

        filtered_code_list = []

        for code in raw_code_list :
            df_kiwoom = kiwoom(code, date)

            if len(df_kiwoom)-timesteps-offset >= 180:
                filtered_code_list.append(code)
    #             print(code)

        df_filtered =df_raw.loc[filtered_code_list]
        df_filtered.to_csv('./csv/filtered/{}_filtered_groupping.csv'.format(date)) 
    
    return df_filtered

def make_group(date):
    
    
    df_group = pd.read_csv('./csv/filtered/{}_filtered_groupping.csv'.format(date), index_col=0)

    B_code_list = df_group[df_group.그룹 == 'B'].index.to_list()
    C_code_list = df_group[df_group.그룹 == 'C'].index.to_list()
    D_code_list = df_group[df_group.그룹 == 'D'].index.to_list()
    E_code_list = df_group[df_group.그룹 == 'E'].index.to_list()

    return B_code_list, C_code_list,D_code_list,E_code_list


def concat_train_test(code_list):
    global code
        
    for i, code in (enumerate(code_list)) :
        
        try :
        
            if i == 0: 
                x_train0, y_train0, x_test0, y_test0 = train_test_spilt()

            else :
                x_train, y_train, x_test, y_test = train_test_spilt()
                
                x_train0 = tf.concat([x_train0, x_train], axis=0)
                y_train0 = tf.concat([y_train0, y_train], axis=0)
                x_test0 = tf.concat([x_test0, x_test], axis=0)
                y_test0 = tf.concat([y_test0, y_test], axis=0)
                
                x_train_concat = x_train0
                y_train_concat = y_train0
                x_test_concat = x_test0
                y_test_concat = y_test0
                
                print('정상', i,'/',len(code_list)-1, code)
                
        except Exception as ex:
            print(ex, code)
            
           
    return x_train_concat, y_train_concat, x_test_concat, y_test_concat 


def load_final_data(path = './numpy/concat/',group = None ,offset=None, timesteps=None):

    dataset_load=np.load(path+'final_{}_dataset_{}_{}.npz'.format(group,timesteps,offset))

    x_train=dataset_load['x_train']
    x_test=dataset_load['x_test']
    y_train=dataset_load['y_train']
    y_test=dataset_load['y_test']

    dataset_load.close()
    
    
    return x_train,x_test,y_train,y_test

def Standardscale(x_train, x_test, y_train, y_test):

    x_train = x_train.reshape(x_train.shape[0],-1)
    x_test = x_test.reshape(x_test.shape[0],-1)
    
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    
    return x_train_scaled,x_test_scaled

In [6]:
def group_precision_score(date):
    
    daesin_kospi(date)
    
    group_list=['B','C','D','E']
    B_code_list, C_code_list,D_code_list,E_code_list = make_group(date)
    
    for group_name, code_list in zip(group_list,[B_code_list, C_code_list, D_code_list,E_code_list]):
        
        global code
        print(group_name, 'group start')  
        
        precision_list= list()
        model = load_model('./model/{}/{}_60_5_final.hdf5'.format(group_name,group_name))
        
        for i, code in enumerate(code_list) : 
            
            daesin_issued(code)
            
            x ,y = split_sequence(timesteps, offset)
            x = x.reshape(x.shape[0],-1)
            e = LabelEncoder()
            y = e.fit_transform(y)
            y= y.astype('float64')

            
            y_pred = np.where(model.predict(x)>0.5, 1,0)
            precision = round(precision_score(y, y_pred),3)
            precision_list.append(precision)
            
            print( i, '/', len(code_list)  , code,   'precision_score : ', precision)
            
        globals()['{}_precision_score'.format(group_name)] = np.mean(precision_list) 
        print(globals()['{}_precision_score'.format(group_name)])
        print()
        print(group_name, 'group done')    
        print()
        
        
    return globals()['{}_precision_score'.format(group_name)]

In [7]:
date = '2021-12-30'
timesteps=60
offset=5
group_precision_score(date)

B group start
0 / 14 a001440 precision_score :  0.747
1 / 14 a004830 precision_score :  0.769
2 / 14 a009680 precision_score :  0.808
3 / 14 a010820 precision_score :  0.66
4 / 14 a011200 precision_score :  0.709
5 / 14 a018470 precision_score :  0.756
6 / 14 a020560 precision_score :  0.737
7 / 14 a024890 precision_score :  0.648
8 / 14 a069620 precision_score :  0.681
9 / 14 a084680 precision_score :  0.613
10 / 14 a090350 precision_score :  0.689
11 / 14 a092200 precision_score :  0.59
12 / 14 a129260 precision_score :  0.698
13 / 14 a383220 precision_score :  0.617
0.6944285714285715

B group done

C group start
0 / 375 a000020 precision_score :  0.725
1 / 375 a000080 precision_score :  0.714
2 / 375 a000100 precision_score :  0.724
3 / 375 a000120 precision_score :  0.75
4 / 375 a000140 precision_score :  0.76
5 / 375 a000150 precision_score :  0.783
6 / 375 a000210 precision_score :  0.686
7 / 375 a000230 precision_score :  0.727
8 / 375 a000240 precision_score :  0.762
9 / 375 a

180 / 375 a016090 precision_score :  0.802
181 / 375 a016360 precision_score :  0.68
182 / 375 a016380 precision_score :  0.781
183 / 375 a016580 precision_score :  0.851
184 / 375 a016740 precision_score :  0.714
185 / 375 a016880 precision_score :  0.731
186 / 375 a017040 precision_score :  0.714
187 / 375 a017550 precision_score :  0.682
188 / 375 a017670 precision_score :  0.68
189 / 375 a017800 precision_score :  0.697
190 / 375 a017810 precision_score :  0.692
191 / 375 a017940 precision_score :  0.876
192 / 375 a017960 precision_score :  0.801
193 / 375 a018260 precision_score :  0.707
194 / 375 a018880 precision_score :  0.744
195 / 375 a019170 precision_score :  0.667
196 / 375 a019180 precision_score :  0.61
197 / 375 a020000 precision_score :  0.698
198 / 375 a020120 precision_score :  0.754
199 / 375 a020150 precision_score :  0.744
200 / 375 a020760 precision_score :  0.763
201 / 375 a021240 precision_score :  0.592
202 / 375 a023530 precision_score :  0.755
203 / 375 a023

372 / 375 a375500 precision_score :  0.752
373 / 375 a377300 precision_score :  0.68
374 / 375 a383800 precision_score :  0.79
0.7105866666666667

C group done

D group start
0 / 144 a000060 precision_score :  0.775
1 / 144 a000180 precision_score :  0.674
2 / 144 a000220 precision_score :  0.573
3 / 144 a000400 precision_score :  0.812
4 / 144 a000890 precision_score :  0.683
5 / 144 a000970 precision_score :  0.794
6 / 144 a001060 precision_score :  0.841
7 / 144 a001200 precision_score :  0.693
8 / 144 a001390 precision_score :  0.7
9 / 144 a001500 precision_score :  0.813
10 / 144 a001570 precision_score :  0.628
11 / 144 a001750 precision_score :  0.788
12 / 144 a001790 precision_score :  0.813
13 / 144 a001820 precision_score :  0.726
14 / 144 a002240 precision_score :  0.785
15 / 144 a002360 precision_score :  0.671
16 / 144 a002410 precision_score :  0.697
17 / 144 a002450 precision_score :  0.812
18 / 144 a002630 precision_score :  0.696
19 / 144 a002710 precision_score :  0.5

0.636875

In [8]:
B_precision_score

0.6944285714285715

In [9]:
C_precision_score

0.7105866666666667

In [12]:
D_precision_score

0.7024722222222223

In [11]:
E_precision_score

0.636875